In [2]:
import pstats
stats = pstats.Stats('02_profiling/profile.stats')
stats.print_stats()

Fri Sep 23 09:12:59 2022    02_profiling/profile.stats

         36221995 function calls in 6.588 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.327    0.327    6.569    6.569 julia1_nopil.py:23(calc_pure_python)
        1    4.165    4.165    6.147    6.147 julia1_nopil.py:9(calculate_z_serial_purepython)
        1    0.019    0.019    6.588    6.588 julia1_nopil.py:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  2002000    0.092    0.000    0.092    0.000 {method 'append' of 'list' objects}
 34219980    1.981    0.000    1.981    0.000 {built-in method builtins.abs}
        1    0.000    0.000    6.588    6.588 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        1    0.004    0.004    0.004    0.004 {

In [3]:
stats.print_callees()

   Random listing order was used

Function                                          called...
                                                      ncalls  tottime  cumtime
julia1_nopil.py:23(calc_pure_python)              ->       1    4.165    6.147  julia1_nopil.py:9(calculate_z_serial_purepython)
                                                           2    0.000    0.000  {built-in method builtins.len}
                                                           3    0.000    0.000  {built-in method builtins.print}
                                                           1    0.004    0.004  {built-in method builtins.sum}
                                                           2    0.000    0.000  {built-in method time.time}
                                                     2002000    0.092    0.092  {method 'append' of 'list' objects}
julia1_nopil.py:9(calculate_z_serial_purepython)  -> 34219980    1.981    1.981  {built-in method builtins.abs}
                           

---

In [4]:
z = 0 + 0j
%timeit abs(z)

28.4 ns ± 0.0875 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


---

In [5]:
from datetime import datetime
from itertools import count, filterfalse, groupby, islice
from random import normalvariate, randint

from scipy.stats import normaltest


def read_fake_data(filename):
    for timestamp in count():
        #  We insert an anomalous data point approximately once a week
        if randint(0, 7 * 60 * 60 * 24 - 1) == 1:
            value = normalvariate(0, 1)
        else:
            value = 100
        yield datetime.fromtimestamp(timestamp), value

data = read_fake_data("fake_filename")

In [104]:
data_list = [next(data) for _ in range(20)]
data_list

[(datetime.datetime(1970, 1, 1, 0, 4, 46), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 47), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 48), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 49), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 50), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 51), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 52), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 53), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 54), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 55), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 56), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 57), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 58), 100),
 (datetime.datetime(1970, 1, 1, 0, 4, 59), 100),
 (datetime.datetime(1970, 1, 1, 0, 5), 100),
 (datetime.datetime(1970, 1, 1, 0, 5, 1), 100),
 (datetime.datetime(1970, 1, 1, 0, 5, 2), 100),
 (datetime.datetime(1970, 1, 1, 0, 5, 3), 100),
 (datetime.datetime(1970, 1, 1, 0, 5, 4), 100),
 (datetime.datetime(1970, 1, 1, 0, 5, 5), 100)]

---

filter generator

In [51]:
from itertools import count, filterfalse, groupby, islice
ff_gen = filterfalse(lambda x: x % 2, range(100))

modify filtered generator

In [52]:
def times_10(gen):
    yield from map(lambda x: x * 10, gen)

get first X indexes of generator

In [65]:
from itertools import count, filterfalse, groupby, islice
evens = filterfalse(lambda x: x % 2, range(100))
#for x in times_10(evens):
for x in islice(times_10(evens), 10):
    print(x)

0
20
40
60
80
100
120
140
160
180


In [68]:
def filter_divisible_by(gen, by=40):
    yield from (x for x in gen if x % 40 == 0)

In [69]:
from itertools import count, filterfalse, groupby, islice
evens = filterfalse(lambda x: x % 2, range(100))
for x in islice(filter_divisible_by(times_10(evens)), 10):
    print(x)

0
40
80
120
160
200
240
280
320
360
